In [1]:
!git clone --depth 1 https://github.com/Zdong104/FNSPID_Financial_News_Dataset.git
%cd FNSPID_Financial_News_Dataset
!ls -la | head -n 20

Cloning into 'FNSPID_Financial_News_Dataset'...
remote: Enumerating objects: 1198, done.
remote: Counting objects: 100% (1198/1198), done.
remote: Compressing objects: 100% (913/913), done.
remote: Total 1198 (delta 279), reused 1162 (delta 275), pack-reused 0 (from 0)
Receiving objects: 100% (1198/1198), 104.88 MiB | 25.02 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Updating files: 100% (1083/1083), done.
/content/FNSPID_Financial_News_Dataset
total 60
drwxr-xr-x  6 root root  4096 Sep  8 01:24 .
drwxr-xr-x  1 root root  4096 Sep  8 01:24 ..
drwxr-xr-x  9 root root  4096 Sep  8 01:24 data_processor
drwxr-xr-x  7 root root  4096 Sep  8 01:24 data_scraper
drwxr-xr-x 15 root root  4096 Sep  8 01:24 dataset_test
-rw-r--r--  1 root root    22 Sep  8 01:24 Download_dataset.py
drwxr-xr-x  8 root root  4096 Sep  8 01:24 .git
-rw-r--r--  1 root root 19490 Sep  8 01:24 LICENSE
-rw-r--r--  1 root root  4692 Sep  8 01:24 README.md
-rw-r--r--  1 root root   245 Sep  8 01:24 requirements.t

In [4]:
!pip -q install pandas==2.2.2 datasets==2.19.1 scikit-learn==1.5.2 requests==2.32.3 tqdm==4.66.4


In [6]:
from datasets import load_dataset
import itertools, pandas as pd

# Stream the first ~5000 rows from the official CSV
url = "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_news/nasdaq_exteral_data.csv"
ds = load_dataset("csv", data_files=url, split="train", streaming=True)

rows = list(itertools.islice(ds, 5000))  # grab a small sample
df = pd.DataFrame(rows)
print(df.shape)
df.head()
df.to_csv("fnspid_news_sample.csv", index=False)


(5000, 12)


In [7]:
#keeping one popular ticker for a clean example
if "stock" in df.columns:
    small = df[df["stock"].astype(str).str.upper().eq("AAPL")].copy()
    if small.empty:
        small = df.sample(min(len(df), 2000), random_state=42).copy()
else:
    small = df.sample(min(len(df), 2000), random_state=42).copy()

small.reset_index(drop=True, inplace=True)
small.to_csv("fnspid_small.csv", index=False)
print(small.shape)
small.head()


(2000, 12)


,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,1501.0,2012-10-11 00:00:00 UTC,Metals & Mining Stock Outlook - Oct. 2012 - In...,AA,https://www.nasdaq.com/articles/metals-mining-...,None,None,The Metals & Mining industry encompasses the e...,The seasonally adjusted annual rate (SAAR) wen...,ALCOA INC (AA): Free Stock Analysis Report BAR...,ALCOA INC (AA): Free Stock Analysis Report BAR...,The seasonally adjusted annual rate (SAAR) wen...
1,2586.0,2023-05-19 00:00:00 UTC,US judge says American-JetBlue must end alliance,AAL,https://www.nasdaq.com/articles/us-judge-says-...,None,None,By Diane Bartz and David Shepardson\nWASHINGTO...,By Diane Bartz and David Shepardson WASHINGTON...,By Diane Bartz and David Shepardson WASHINGTON...,By Diane Bartz and David Shepardson WASHINGTON...,By Diane Bartz and David Shepardson WASHINGTON...
2,2653.0,2023-04-21 00:00:00 UTC,"SQM, Albemarle shares slide on Chile lithium n...",AAL,https://www.nasdaq.com/articles/sqm-albemarle-...,None,None,By Alexander Villegas and Ernest Scheyder\nSAN...,Shares in rival Anglo American AAL.L fell 4% o...,Shares in rival Anglo American AAL.L fell 4% o...,Shares in rival Anglo American AAL.L fell 4% o...,Shares in rival Anglo American AAL.L fell 4% o...
3,1055.0,2021-04-15 00:00:00 UTC,Alcoa Q1 Profit Trumps Street View As Aluminum...,AA,https://www.nasdaq.com/articles/alcoa-q1-profi...,None,None,"(RTTNews) - Alcoa Corp. (AA), the largest prod...","(RTTNews) - Alcoa Corp. (AA), the largest prod...","(RTTNews) - Alcoa Corp. (AA), the largest prod...","(RTTNews) - Alcoa Corp. (AA), the largest prod...","(RTTNews) - Alcoa Corp. (AA), the largest prod..."
4,705.0,2022-07-14 00:00:00 UTC,Should iShares Core S&P MidCap ETF (IJH) Be on...,AA,https://www.nasdaq.com/articles/should-ishares...,None,None,Looking for broad exposure to the Mid Cap Blen...,"Looking at individual holdings, Molina Healthc...","Looking at individual holdings, Molina Healthc...","Looking at individual holdings, Molina Healthc...",Alcoa (AA): Free Stock Analysis Report Looking...


In [8]:
from sklearn.model_selection import train_test_split

base = small.copy()
train_df, temp = train_test_split(base, test_size=0.30, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp, test_size=(2/3), random_state=42, shuffle=True)

print("Train/Test/Val sizes:", len(train_df), len(test_df), len(val_df))

train_df.to_csv("news_train.csv", index=False)
test_df.to_csv("news_test.csv", index=False)
val_df.to_csv("news_val.csv", index=False)


Train/Test/Val sizes: 1400 400 200


In [9]:
!zip -q fnspid_artifacts.zip fnspid_news_sample.csv fnspid_small.csv news_train.csv news_test.csv news_val.csv
